In [ ]:
# Full GA with monorail (5 devices), 3 mills, 2 hoists, 32 ovens, and detailed constraints.
# Retry execution with slightly reduced GA size to avoid timeouts.
import random, time
from dataclasses import dataclass
from typing import List, Dict, Optional, Tuple
import pandas as pd

# ---------- Configurable parameters (replace with your real values) ----------
HORIZON = 24 * 360  # minutes
CURING_TIME = 300  # fixed oven time in seconds
NUM_OVENS = 32
OVENS_PER_ROW = 16
NUM_HOISTS = 2
NUM_MILLS = 3
NUM_DEVICES = 5  # rolling devices on monorail
HOIST_OP_DURATION = 6  # load+unload time

# Rolling times per size (minutes) r1,r2,r3
ROLLING_TIME = {
    'S': (23, 18, 26),
    'M': (28, 29, 36),
    'L': (30, 42, 47)
}

# Monorail segment times (minutes) - placeholders; adjust to your layout distances
SEG_TIME = {
    'M1_M2': 35,
    'M2_M3': 32,
    'M3_HA': 20,
    'M3_HB': 26,
    'HA_M1': 15,
    'HB_M1': 15
}

MOVE_INTERNAL = {
    'after_m1': 1,
    'after_m2': 1,
    'after_m3': 1
}

OVENS_BY_PID = {
    'A': list(range(0, 10)),
    'B': list(range(10, 20)),
    'C': list(range(20, 32))
}

BASE_MOVE_TO_OVEN = 3
def move_time_to_oven(oven_id: int) -> int:
    pos = oven_id % OVENS_PER_ROW
    return BASE_MOVE_TO_OVEN + (pos % 4)

def hoist_for_oven(oven_id: int) -> int:
    return 0 if oven_id < OVENS_PER_ROW else 1

@dataclass
class Job:
    id: int
    pid: str
    size: str

@dataclass
class Interval:
    start: int
    end: int  # exclusive

class BusyCalendar:
    def __init__(self):
        self.intervals: List[Interval] = []

    def is_free(self, start: int, duration: int) -> bool:
        end = start + duration
        if start < 0 or end > HORIZON:
            return False
        for it in self.intervals:
            if it.start >= end:
                break
            if it.end <= start:
                continue
            return False
        return True

    def occupy(self, start: int, duration: int):
        end = start + duration
        new = Interval(start, end)
        res = []
        placed = False
        for it in self.intervals:
            if it.end < new.start:
                res.append(it)
            elif it.start > new.end:
                if not placed:
                    res.append(new)
                    placed = True
                res.append(it)
            else:
                new.start = min(new.start, it.start)
                new.end = max(new.end, it.end)
        if not placed:
            res.append(new)
        self.intervals = sorted(res, key=lambda x: x.start)

    def earliest_free(self, earliest_start: int, duration: int) -> Optional[int]:
        if duration <= 0:
            return earliest_start if earliest_start <= HORIZON else None
        if earliest_start < 0: earliest_start = 0
        if not self.intervals:
            if earliest_start + duration <= HORIZON:
                return earliest_start
            return None
        if earliest_start + duration <= self.intervals[0].start:
            return earliest_start
        for i in range(len(self.intervals)-1):
            gap_start = max(earliest_start, self.intervals[i].end)
            gap_end = self.intervals[i+1].start
            if gap_start + duration <= gap_end:
                return gap_start
        gap_start = max(earliest_start, self.intervals[-1].end)
        if gap_start + duration <= HORIZON:
            return gap_start
        return None

def decode_chromosome(chromosome: List[int], jobs: Dict[int, Job]) -> Tuple[int, Dict[int, Dict]]:
    mills = [BusyCalendar() for _ in range(NUM_MILLS)]
    segments = {name: BusyCalendar() for name in SEG_TIME.keys()}
    hoists = [BusyCalendar() for _ in range(NUM_HOISTS)]
    ovens = [BusyCalendar() for _ in range(NUM_OVENS)]

    device_time = [0 for _ in range(NUM_DEVICES)]
    next_job_ptr = 0
    total_jobs = len(chromosome)
    schedule = {jid: {'assigned': False} for jid in jobs.keys()}
    completed = 0

    while next_job_ptr < total_jobs:
        d_idx = min(range(NUM_DEVICES), key=lambda d: device_time[d])
        current_t = device_time[d_idx]
        if current_t >= HORIZON:
            break
        job_id = chromosome[next_job_ptr]
        job = jobs[job_id]
        t = current_t
        r1, r2, r3 = ROLLING_TIME[job.size]

        # find t1
        t1 = mills[0].earliest_free(t, r1)
        if t1 is None:
            schedule[job_id]['assigned'] = False
            next_job_ptr += 1
            next_free = mills[0].earliest_free(t, 1)
            device_time[d_idx] = next_free if next_free is not None else HORIZON
            continue
        t1_end = t1 + r1
        mills[0].occupy(t1, r1)

        # seg M1_M2
        seg = 'M1_M2'; seg_time = SEG_TIME[seg]
        seg_start = segments[seg].earliest_free(t1_end + MOVE_INTERNAL['after_m1'], seg_time)
        if seg_start is None:
            # rollback mill1
            if mills[0].intervals and mills[0].intervals[-1].start == t1:
                mills[0].intervals.pop()
            schedule[job_id]['assigned'] = False
            next_job_ptr += 1
            device_time[d_idx] = t1_end + 1
            continue
        seg_end = seg_start + seg_time
        segments[seg].occupy(seg_start, seg_time)

        # mill2
        t2 = mills[1].earliest_free(seg_end + MOVE_INTERNAL['after_m2'], r2)
        if t2 is None:
            # rollback
            if segments[seg].intervals and segments[seg].intervals[-1].start == seg_start:
                segments[seg].intervals.pop()
            if mills[0].intervals and mills[0].intervals[-1].start == t1:
                mills[0].intervals.pop()
            schedule[job_id]['assigned'] = False
            next_job_ptr += 1
            device_time[d_idx] = seg_end + 1
            continue
        t2_end = t2 + r2
        mills[1].occupy(t2, r2)

        # seg M2_M3
        seg2 = 'M2_M3'; seg2_time = SEG_TIME[seg2]
        seg2_start = segments[seg2].earliest_free(t2_end + MOVE_INTERNAL['after_m2'], seg2_time)
        if seg2_start is None:
            # rollback mills and seg
            if mills[1].intervals and mills[1].intervals[-1].start == t2:
                mills[1].intervals.pop()
            if segments[seg].intervals and segments[seg].intervals[-1].start == seg_start:
                segments[seg].intervals.pop()
            if mills[0].intervals and mills[0].intervals[-1].start == t1:
                mills[0].intervals.pop()
            schedule[job_id]['assigned'] = False
            next_job_ptr += 1
            device_time[d_idx] = t2_end + 1
            continue
        seg2_end = seg2_start + seg2_time
        segments[seg2].occupy(seg2_start, seg2_time)

        # mill3
        t3 = mills[2].earliest_free(seg2_end + MOVE_INTERNAL['after_m3'], r3)
        if t3 is None:
            # rollback
            if segments[seg2].intervals and segments[seg2].intervals[-1].start == seg2_start:
                segments[seg2].intervals.pop()
            if mills[1].intervals and mills[1].intervals[-1].start == t2:
                mills[1].intervals.pop()
            if segments[seg].intervals and segments[seg].intervals[-1].start == seg_start:
                segments[seg].intervals.pop()
            if mills[0].intervals and mills[0].intervals[-1].start == t1:
                mills[0].intervals.pop()
            schedule[job_id]['assigned'] = False
            next_job_ptr += 1
            device_time[d_idx] = seg2_end + 1
            continue
        t3_end = t3 + r3
        mills[2].occupy(t3, r3)

        # choose oven among PID ovens
        candidate_ovens = OVENS_BY_PID.get(job.pid, [])
        chosen = None
        for oven_id in candidate_ovens:
            hoist_idx = hoist_for_oven(oven_id)
            seg_name = 'M3_HA' if hoist_idx == 0 else 'M3_HB'
            seg3_time = SEG_TIME[seg_name]
            seg3_start = segments[seg_name].earliest_free(t3_end + MOVE_INTERNAL['after_m3'], seg3_time)
            if seg3_start is None:
                continue
            seg3_end = seg3_start + seg3_time
            hoist_ready = hoists[hoist_idx].earliest_free(seg3_end, HOIST_OP_DURATION)
            if hoist_ready is None:
                continue
            move_to_oven = move_time_to_oven(oven_id)
            oven_start_candidate = max(seg3_end + move_to_oven, hoist_ready + HOIST_OP_DURATION)
            if oven_start_candidate + CURING_TIME <= HORIZON and ovens[oven_id].is_free(oven_start_candidate, CURING_TIME):
                chosen = (oven_id, seg_name, seg3_start, hoist_idx, hoist_ready, oven_start_candidate)
                break

        if chosen is None:
            # rollback mills and segments
            if mills[2].intervals and mills[2].intervals[-1].start == t3:
                mills[2].intervals.pop()
            if segments['M2_M3'].intervals and segments['M2_M3'].intervals[-1].start == seg2_start:
                segments['M2_M3'].intervals.pop()
            if mills[1].intervals and mills[1].intervals[-1].start == t2:
                mills[1].intervals.pop()
            if segments['M1_M2'].intervals and segments['M1_M2'].intervals[-1].start == seg_start:
                segments['M1_M2'].intervals.pop()
            if mills[0].intervals and mills[0].intervals[-1].start == t1:
                mills[0].intervals.pop()
            schedule[job_id]['assigned'] = False
            next_job_ptr += 1
            device_time[d_idx] = t3_end + 1
            continue

        oven_id, seg3_name, seg3_start, hoist_idx, hoist_ready, oven_start = chosen
        seg3_dur = SEG_TIME[seg3_name]
        segments[seg3_name].occupy(seg3_start, seg3_dur)
        hoists[hoist_idx].occupy(hoist_ready, HOIST_OP_DURATION)
        ovens[oven_id].occupy(oven_start, CURING_TIME)

        # return segment from hoist to M1
        ret_seg = 'HA_M1' if hoist_idx==0 else 'HB_M1'
        ret_dur = SEG_TIME[ret_seg]
        ret_start = segments[ret_seg].earliest_free(hoist_ready + HOIST_OP_DURATION, ret_dur)
        if ret_start is None:
            # schedule at earliest possible (use earliest_free again)
            ret_start = segments[ret_seg].earliest_free(hoist_ready + HOIST_OP_DURATION, ret_dur)
            if ret_start is None:
                device_time[d_idx] = HORIZON
            else:
                segments[ret_seg].occupy(ret_start, ret_dur)
                device_time[d_idx] = ret_start + ret_dur
        else:
            segments[ret_seg].occupy(ret_start, ret_dur)
            device_time[d_idx] = ret_start + ret_dur

        schedule[job_id] = {
            'assigned': True, 'device': d_idx,
            'mill1_start': t1, 'mill1_end': t1_end,
            'mill2_start': t2, 'mill2_end': t2_end,
            'mill3_start': t3, 'mill3_end': t3_end,
            'seg_M1_M2_start': seg_start, 'seg_M1_M2_end': seg_end,
            'seg_M2_M3_start': seg2_start, 'seg_M2_M3_end': seg2_end,
            'seg_M3_H_start': seg3_start, 'seg_M3_H_end': seg3_start+seg3_dur,
            'hoist_idx': hoist_idx, 'hoist_start': hoist_ready,
            'oven_id': oven_id, 'oven_start': oven_start, 'oven_end': oven_start + CURING_TIME
        }

        if schedule[job_id]['oven_end'] <= HORIZON:
            completed += 1
        next_job_ptr += 1

    return completed, schedule

# ---------- GA functions ----------
def generate_jobs(n: int, pids: List[str]=None) -> Dict[int, Job]:
    if pids is None: pids = list(OVENS_BY_PID.keys())
    sizes = list(ROLLING_TIME.keys())
    jobs = {}
    for i in range(n):
        pid = random.choice(pids)
        size = random.choices(sizes, weights=[0.4,0.4,0.2])[0]
        jobs[i] = Job(i, pid, size)
    return jobs

def initial_population(job_ids: List[int], pop_size: int):
    pop = []
    sorted_ids = sorted(job_ids, key=lambda j: sum(ROLLING_TIME[jobs[j].size]))
    pop.append(sorted_ids.copy())
    for _ in range(pop_size-1):
        indiv = job_ids.copy(); random.shuffle(indiv); pop.append(indiv)
    return pop

def tournament_select(pop, fits, k=3):
    idxs = random.sample(range(len(pop)), k)
    best = max(idxs, key=lambda i: fits[i])
    return pop[best].copy()

def pmx(p1,p2):
    n=len(p1); a,b=sorted(random.sample(range(n),2))
    child=[None]*n; child[a:b+1]=p1[a:b+1]
    for i in range(a,b+1):
        if p2[i] not in child:
            val=p2[i]; pos=i
            while True:
                val_in_p1=p1[pos]; pos=p2.index(val_in_p1)
                if child[pos] is None:
                    child[pos]=p2[i]; break
    for i in range(n):
        if child[i] is None: child[i]=p2[i]
    return child

def swap_mut(indiv, rate=0.2):
    if random.random() < rate:
        i,j=random.sample(range(len(indiv)),2); indiv[i],indiv[j]=indiv[j],indiv[i]
    return indiv

def run_ga(jobs: Dict[int, Job], pop_size=30, generations=40, cx_rate=0.8, mut_rate=0.2):
    job_ids=list(jobs.keys())
    pop=initial_population(job_ids, pop_size)
    best_fit=-1; best_ind=None; best_sched=None; history=[]
    for gen in range(generations):
        fits=[]; scheds=[]
        for indiv in pop:
            fit,sched = decode_chromosome(indiv, jobs)
            fits.append(fit); scheds.append(sched)
            if fit>best_fit:
                best_fit=fit; best_ind=indiv.copy(); best_sched=sched
        history.append(max(fits))
        new_pop=[pop[max(range(len(pop)), key=lambda i: fits[i])].copy()]
        while len(new_pop)<pop_size:
            p1=tournament_select(pop,fits); p2=tournament_select(pop,fits)
            if random.random()<cx_rate:
                c1=pmx(p1,p2); c2=pmx(p2,p1)
            else:
                c1=p1.copy(); c2=p2.copy()
            c1=swap_mut(c1,mut_rate); c2=swap_mut(c2,mut_rate)
            new_pop.extend([c1,c2])
        pop=new_pop[:pop_size]
        if (gen+1)%10==0 or gen==0:
            print(f"Gen {gen+1} best_completed={best_fit}")
    return best_fit, best_ind, best_sched, history

# ---------- Demo ----------
random.seed(1234)
NUM_JOBS = 1200
jobs = generate_jobs(NUM_JOBS)
start=time.time()
best_fit,best_ind,best_sched,history = run_ga(jobs, pop_size=35, generations=45, cx_rate=0.85, mut_rate=0.18)
elapsed=time.time()-start
print(f"\nGA done in {elapsed:.1f}s. Best completed tires: {best_fit} / {len(jobs)}")

# # Prepare hourly summary and sample scheduled jobs
# completed_ends = []
# for jid,info in best_sched.items():
#     if info.get('assigned') and info.get('oven_end') is not None and info.get('oven_end')<=HORIZON:
#         completed_ends.append(info['oven_end'])
# hourly=[0]*24
# for t in completed_ends:
#     h=min(23,t//60); hourly[h]+=1
# df=pd.DataFrame({'hour':list(range(24)),'completed':hourly,'cumulative':pd.Series(hourly).cumsum()})
# import caas_jupyter_tools as cjt; cjt.display_dataframe_to_user("Hourly Completed Tires", df)

# sched_list=[]
# for jid,info in best_sched.items():
#     if info.get('assigned') and info.get('oven_end') is not None:
#         sched_list.append({'job_id':jid,'pid':jobs[jid].pid,'size':jobs[jid].size,'device':info['device'],'oven_id':info['oven_id'],'oven_start':info['oven_start'],'oven_end':info['oven_end']})
# sched_df=pd.DataFrame(sched_list).sort_values('oven_start').head(60)
# cjt.display_dataframe_to_user("Sample Scheduled Jobs", sched_df)

# print("\nReplace timing parameters and OVENS_BY_PID with your real data to re-run.")


Gen 1 best_completed=223
Gen 10 best_completed=224
Gen 20 best_completed=224
Gen 30 best_completed=224
Gen 40 best_completed=224

GA done in 139.7s. Best completed tires: 224 / 1000


In [19]:
# tire_ga.py
# Genetic Algorithm for tire scheduling using oven-job permutation encoding.
# Copy and run with Python 3.8+.

import random
import time
from dataclasses import dataclass
from typing import List, Dict, Optional, Tuple

# ---------------------- CONFIGURATION (from your data) ----------------------
HORIZON = 24 * 3600          # 24 hours in seconds
CURING_TIME = 3000            # 300 seconds for curing
NUM_OVENS = 32
OVENS_PER_ROW = 16
NUM_HOISTS = 2
NUM_MILLS = 3
NUM_DEVICES = 5              # five rolling devices on monorail
HOIST_OP_DURATION = 6        # hoist load/unload operation (seconds) - adjust if needed

# Oven -> (PID, hoist_travel_time_seconds)
# (From your table: Oven 1..32; traveling times and PID)
OVEN_INFO = {
    1:  ('A', 10),  2: ('C', 12),  3: ('G', 14),  4: ('E', 16),
    5:  ('B', 18),  6: ('D', 20),  7: ('F', 22),  8: ('H', 24),
    9:  ('A', 26), 10: ('D', 28), 11: ('G', 30), 12: ('F', 32),
    13: ('H', 34), 14: ('G', 36), 15: ('F', 38), 16: ('A', 40),
    17: ('E', 10), 18: ('D', 12), 19: ('E', 14), 20: ('A', 16),
    21: ('G', 18), 22: ('G', 20), 23: ('D', 22), 24: ('B', 24),
    25: ('F', 26), 26: ('H', 28), 27: ('A', 30), 28: ('E', 32),
    29: ('B', 34), 30: ('H', 36), 31: ('D', 38), 32: ('G', 40),
}

# Mill processing times per PID (seconds) r1, r2, r3
MILL_TIMES = {
    'A': (21, 19, 28),
    'B': (18, 20, 22),
    'C': (26, 24, 31),
    'D': (20, 23, 26),
    'E': (24, 23, 30),
    'F': (15, 13, 17),
    'G': (17, 18, 22),
    'H': (31, 26, 38)
}

# Monorail / segment traveling times (seconds) - from your table
SEG_TIME = {
    'M1_M2': 5,
    'M2_M3': 4,
    'M3_CART': 8,   # M3 to cart/hoist path
    'CART_M1': 3,   # cart back to Mill1 path (device return)
    # We'll use M3_CART and then hoist-specific small travel = oven table value
}

# small internal move delays (can be zero)
MOVE_INTERNAL = {
    'after_m1': 0,
    'after_m2': 0,
    'after_m3': 0
}

# mapping oven -> hoist index (0 or 1)
def hoist_for_oven(oven_id: int) -> int:
    return 0 if oven_id <= OVENS_PER_ROW else 1

# ---------------------- DATA STRUCTURES ----------------------
@dataclass
class Interval:
    start: int
    end: int  # exclusive

class BusyCalendar:
    """Simple interval calendar for exclusive resource usage."""
    def __init__(self):
        self.intervals: List[Interval] = []

    def is_free(self, start: int, dur: int) -> bool:
        end = start + dur
        if start < 0 or end > HORIZON:
            return False
        for it in self.intervals:
            if it.start >= end:
                break
            if it.end <= start:
                continue
            return False
        return True

    def occupy(self, start: int, dur: int):
        end = start + dur
        new = Interval(start, end)
        res: List[Interval] = []
        placed = False
        for it in self.intervals:
            if it.end < new.start:
                res.append(it)
            elif it.start > new.end:
                if not placed:
                    res.append(new)
                    placed = True
                res.append(it)
            else:
                new.start = min(new.start, it.start)
                new.end = max(new.end, it.end)
        if not placed:
            res.append(new)
        self.intervals = sorted(res, key=lambda x: x.start)

    def earliest_free(self, earliest_start: int, dur: int) -> Optional[int]:
        """Return earliest t >= earliest_start where [t,t+dur) is free or None."""
        if dur <= 0:
            return earliest_start if earliest_start <= HORIZON else None
        if earliest_start < 0:
            earliest_start = 0
        if not self.intervals:
            return earliest_start if earliest_start + dur <= HORIZON else None
        # check before first
        if earliest_start + dur <= self.intervals[0].start:
            return earliest_start
        # scan gaps
        for i in range(len(self.intervals) - 1):
            gap_start = max(earliest_start, self.intervals[i].end)
            gap_end = self.intervals[i+1].start
            if gap_start + dur <= gap_end:
                return gap_start
        # after last
        gap_start = max(earliest_start, self.intervals[-1].end)
        if gap_start + dur <= HORIZON:
            return gap_start
        return None

# ---------------------- DECODER ----------------------
def decode_permutation(perm: List[int]) -> Tuple[int, Dict[int, dict]]:
    """
    perm: list of oven IDs in ordering (1..32)
    Returns: (completed_count, schedule_dict)
    schedule_dict[job_oven] has assigned times and resource info.
    The decoder enforces hoist alternation by selecting a next job with opposite hoist where possible.
    """
    # resources
    mills = [BusyCalendar() for _ in range(NUM_MILLS)]          # Mill1, Mill2, Mill3
    segments = {
        'M1_M2': BusyCalendar(),
        'M2_M3': BusyCalendar(),
        'M3_CART': BusyCalendar(),
        'CART_M1': BusyCalendar()
    }
    hoists = [BusyCalendar() for _ in range(NUM_HOISTS)]
    ovens = [BusyCalendar() for _ in range(NUM_OVENS + 1)]     # 1-index oven calendars

    # device available times (when device returns to Mill1 and ready)
    device_ready = [0 for _ in range(NUM_DEVICES)]
    # pointer for permutation
    scheduled = {oven: {'assigned': False} for oven in range(1, NUM_OVENS + 1)}
    completed = 0

    # last hoist used (for alternation). Initialize to None so first can be either.
    last_hoist_used: Optional[int] = None

    # We'll maintain a pointer index into perm; but implement lookahead for alternation:
    perm_list = perm.copy()
    ptr = 0

    while ptr < len(perm_list):
        # pick the device that becomes free the earliest
        d = min(range(NUM_DEVICES), key=lambda i: device_ready[i])
        current_time = device_ready[d]
        if current_time >= HORIZON:
            break

        # fetch the next job respecting alternation: try to find next perm entry whose hoist != last_hoist_used
        chosen_index = None
        chosen_oven = None
        # Lookahead window: up to L entries ahead to find opposite hoist. (TUNE as necessary)
        LOOKAHEAD = 10
        for look in range(LOOKAHEAD):
            idx = ptr + look
            if idx >= len(perm_list):
                break
            cand = perm_list[idx]
            if scheduled[cand]['assigned']:
                continue
            cand_hoist = hoist_for_oven(cand)
            if last_hoist_used is None or cand_hoist != last_hoist_used:
                chosen_index = idx
                chosen_oven = cand
                break
        # if not found, take ptr
        if chosen_index is None:
            # find next unassigned
            idx = ptr
            while idx < len(perm_list) and scheduled[perm_list[idx]]['assigned']:
                idx += 1
            if idx >= len(perm_list):
                break
            chosen_index = idx
            chosen_oven = perm_list[chosen_index]

        # remove selected oven from its position by swapping with ptr (to consume)
        perm_list[chosen_index], perm_list[ptr] = perm_list[ptr], perm_list[chosen_index]
        oven_id = perm_list[ptr]

        pid, hoist_travel = OVEN_INFO[oven_id]
        # begin scheduling the job for device d starting at >= current_time
        # Step 1: Mill1
        r1, r2, r3 = MILL_TIMES[pid]
        t1 = mills[0].earliest_free(current_time, r1)
        if t1 is None:
            # cannot schedule; mark unassigned and advance ptr to next
            scheduled[oven_id]['assigned'] = False
            ptr += 1
            # advance device to next small time to avoid infinite loop
            device_ready[d] = current_time + 1
            continue
        mills[0].occupy(t1, r1)
        t1_end = t1 + r1

        # M1 -> M2 segment
        seg1_start = segments['M1_M2'].earliest_free(t1_end + MOVE_INTERNAL['after_m1'], SEG_TIME['M1_M2'])
        if seg1_start is None:
            # rollback Mill1 occupancy (simple rollback by removing last interval if matches)
            if mills[0].intervals and mills[0].intervals[-1].start == t1 and mills[0].intervals[-1].end == t1_end:
                mills[0].intervals.pop()
            scheduled[oven_id]['assigned'] = False
            ptr += 1
            device_ready[d] = t1_end + 1
            continue
        segments['M1_M2'].occupy(seg1_start, SEG_TIME['M1_M2'])
        seg1_end = seg1_start + SEG_TIME['M1_M2']

        # Mill2
        t2 = mills[1].earliest_free(seg1_end + MOVE_INTERNAL['after_m2'], r2)
        if t2 is None:
            # rollback
            # remove last seg and mill1 if they match
            if segments['M1_M2'].intervals and segments['M1_M2'].intervals[-1].start == seg1_start:
                segments['M1_M2'].intervals.pop()
            if mills[0].intervals and mills[0].intervals[-1].start == t1:
                mills[0].intervals.pop()
            scheduled[oven_id]['assigned'] = False
            ptr += 1
            device_ready[d] = seg1_end + 1
            continue
        mills[1].occupy(t2, r2)
        t2_end = t2 + r2

        # M2 -> M3
        seg2_start = segments['M2_M3'].earliest_free(t2_end + MOVE_INTERNAL['after_m2'], SEG_TIME['M2_M3'])
        if seg2_start is None:
            # rollback
            if segments['M1_M2'].intervals and segments['M1_M2'].intervals[-1].start == seg1_start:
                segments['M1_M2'].intervals.pop()
            if mills[0].intervals and mills[0].intervals[-1].start == t1:
                mills[0].intervals.pop()
            if mills[1].intervals and mills[1].intervals[-1].start == t2:
                mills[1].intervals.pop()
            scheduled[oven_id]['assigned'] = False
            ptr += 1
            device_ready[d] = t2_end + 1
            continue
        segments['M2_M3'].occupy(seg2_start, SEG_TIME['M2_M3'])
        seg2_end = seg2_start + SEG_TIME['M2_M3']

        # Mill3
        t3 = mills[2].earliest_free(seg2_end + MOVE_INTERNAL['after_m3'], r3)
        if t3 is None:
            # rollback
            if segments['M2_M3'].intervals and segments['M2_M3'].intervals[-1].start == seg2_start:
                segments['M2_M3'].intervals.pop()
            if mills[1].intervals and mills[1].intervals[-1].start == t2:
                mills[1].intervals.pop()
            if segments['M1_M2'].intervals and segments['M1_M2'].intervals[-1].start == seg1_start:
                segments['M1_M2'].intervals.pop()
            if mills[0].intervals and mills[0].intervals[-1].start == t1:
                mills[0].intervals.pop()
            scheduled[oven_id]['assigned'] = False
            ptr += 1
            device_ready[d] = seg2_end + 1
            continue
        mills[2].occupy(t3, r3)
        t3_end = t3 + r3

        # M3 -> CART (segment)
        seg3_start = segments['M3_CART'].earliest_free(t3_end, SEG_TIME['M3_CART'])
        if seg3_start is None:
            # rollback mill3 etc.
            if mills[2].intervals and mills[2].intervals[-1].start == t3:
                mills[2].intervals.pop()
            if segments['M2_M3'].intervals and segments['M2_M3'].intervals[-1].start == seg2_start:
                segments['M2_M3'].intervals.pop()
            if mills[1].intervals and mills[1].intervals[-1].start == t2:
                mills[1].intervals.pop()
            if segments['M1_M2'].intervals and segments['M1_M2'].intervals[-1].start == seg1_start:
                segments['M1_M2'].intervals.pop()
            if mills[0].intervals and mills[0].intervals[-1].start == t1:
                mills[0].intervals.pop()
            scheduled[oven_id]['assigned'] = False
            ptr += 1
            device_ready[d] = t3_end + 1
            continue
        segments['M3_CART'].occupy(seg3_start, SEG_TIME['M3_CART'])
        seg3_end = seg3_start + SEG_TIME['M3_CART']

        # Hoist segment: from cart to hoist (we model with hoist resource and oven-specific travel)
        hoist_idx = hoist_for_oven(oven_id)
        # hoist travel time from cart to oven is given by oven table; we assumed cart->hoist segment ends before hoist op
        # Check hoist availability at seg3_end (device arrives)
        hoist_available = hoists[hoist_idx].earliest_free(seg3_end, HOIST_OP_DURATION)
        if hoist_available is None:
            # rollback many resources; simplified: rollback last few segments and mills
            if segments['M3_CART'].intervals and segments['M3_CART'].intervals[-1].start == seg3_start:
                segments['M3_CART'].intervals.pop()
            if mills[2].intervals and mills[2].intervals[-1].start == t3:
                mills[2].intervals.pop()
            if segments['M2_M3'].intervals and segments['M2_M3'].intervals[-1].start == seg2_start:
                segments['M2_M3'].intervals.pop()
            if mills[1].intervals and mills[1].intervals[-1].start == t2:
                mills[1].intervals.pop()
            if segments['M1_M2'].intervals and segments['M1_M2'].intervals[-1].start == seg1_start:
                segments['M1_M2'].intervals.pop()
            if mills[0].intervals and mills[0].intervals[-1].start == t1:
                mills[0].intervals.pop()
            scheduled[oven_id]['assigned'] = False
            ptr += 1
            device_ready[d] = seg3_end + 1
            continue

        # Occupy hoist and oven
        hoists[hoist_idx].occupy(hoist_available, HOIST_OP_DURATION)
        # oven move time (cart->oven hoist travel) from table:
        oven_travel = OVEN_INFO[oven_id][1]
        oven_insert_time = max(seg3_end + oven_travel, hoist_available + HOIST_OP_DURATION)
        # Check oven availability
        if oven_insert_time + CURING_TIME > HORIZON or not ovens[oven_id].is_free(oven_insert_time, CURING_TIME):
            # rollback hoist, segs, mills
            # remove hoist last interval
            if hoists[hoist_idx].intervals and hoists[hoist_idx].intervals[-1].start == hoist_available:
                hoists[hoist_idx].intervals.pop()
            if segments['M3_CART'].intervals and segments['M3_CART'].intervals[-1].start == seg3_start:
                segments['M3_CART'].intervals.pop()
            if mills[2].intervals and mills[2].intervals[-1].start == t3:
                mills[2].intervals.pop()
            if segments['M2_M3'].intervals and segments['M2_M3'].intervals[-1].start == seg2_start:
                segments['M2_M3'].intervals.pop()
            if mills[1].intervals and mills[1].intervals[-1].start == t2:
                mills[1].intervals.pop()
            if segments['M1_M2'].intervals and segments['M1_M2'].intervals[-1].start == seg1_start:
                segments['M1_M2'].intervals.pop()
            if mills[0].intervals and mills[0].intervals[-1].start == t1:
                mills[0].intervals.pop()
            scheduled[oven_id]['assigned'] = False
            ptr += 1
            device_ready[d] = oven_insert_time + 1
            continue

        ovens[oven_id].occupy(oven_insert_time, CURING_TIME)

        # Occupy return segment from cart/hoist back to Mill1 for the device
        ret_seg_start = segments['CART_M1'].earliest_free(hoist_available + HOIST_OP_DURATION, SEG_TIME['CART_M1'])
        if ret_seg_start is None:
            # if cannot find a return slot we schedule when available; for simplicity take earliest possible
            ret_seg_start = segments['CART_M1'].earliest_free(hoist_available + HOIST_OP_DURATION, SEG_TIME['CART_M1'])
            if ret_seg_start is None:
                device_ready[d] = HORIZON
            else:
                segments['CART_M1'].occupy(ret_seg_start, SEG_TIME['CART_M1'])
                device_ready[d] = ret_seg_start + SEG_TIME['CART_M1']
        else:
            segments['CART_M1'].occupy(ret_seg_start, SEG_TIME['CART_M1'])
            device_ready[d] = ret_seg_start + SEG_TIME['CART_M1']

        scheduled[oven_id] = {
            'assigned': True,
            'device': d,
            'mill1': (t1, t1_end),
            'm1m2_seg': (seg1_start, seg1_end),
            'mill2': (t2, t2_end),
            'm2m3_seg': (seg2_start, seg2_end),
            'mill3': (t3, t3_end),
            'm3cart_seg': (seg3_start, seg3_end),
            'hoist_idx': hoist_idx,
            'hoist_start': hoist_available,
            'oven_start': oven_insert_time,
            'oven_end': oven_insert_time + CURING_TIME
        }

        if scheduled[oven_id]['oven_end'] <= HORIZON:
            completed += 1

        # update last_hoist_used and increment ptr to consume job
        last_hoist_used = hoist_idx
        ptr += 1
    return completed, scheduled


# ---------------------- GA (permutation based) ----------------------
def generate_initial_permutations(seed: int = 42) -> List[List[int]]:
    random.seed(seed)
    base = list(range(1, NUM_OVENS + 1))
    pop = []
    # heuristic: interleaved to satisfy alternation seed
    interleaved = []
    for i in range(OVENS_PER_ROW):
        interleaved.append(i + 1)
        interleaved.append(i + 1 + OVENS_PER_ROW)
    pop.append(interleaved[:])  # good seed
    for _ in range(1, 30):
        tmp = base[:]
        random.shuffle(tmp)
        pop.append(tmp)
    return pop

def tournament_select(pop: List[List[int]], fits: List[int], k=3) -> List[int]:
    idxs = random.sample(range(len(pop)), k)
    best = max(idxs, key=lambda i: fits[i])
    return pop[best][:]

def pmx_crossover(p1: List[int], p2: List[int]) -> Tuple[List[int], List[int]]:
    n = len(p1)
    a, b = sorted(random.sample(range(n), 2))
    def pmx(parent_a, parent_b):
        child = [None]*n
        child[a:b+1] = parent_a[a:b+1]
        for i in range(a, b+1):
            if parent_b[i] not in child:
                val = parent_b[i]; pos = i
                while True:
                    val_in_a = parent_a[pos]
                    pos = parent_b.index(val_in_a)
                    if child[pos] is None:
                        child[pos] = parent_b[i]
                        break
        for i in range(n):
            if child[i] is None:
                child[i] = parent_b[i]
        return child
    return pmx(p1, p2), pmx(p2, p1)

def swap_mutation(indiv: List[int], rate=0.2) -> List[int]:
    if random.random() < rate:
        i, j = random.sample(range(len(indiv)), 2)
        indiv[i], indiv[j] = indiv[j], indiv[i]
    return indiv

def run_ga(pop_size=35, generations=60):
    # prepare jobs: ovens 1..32
    initial_pop = generate_initial_permutations()
    pop = initial_pop[:pop_size]
    print(f"Initial population size: {len(pop)}")
    best_fit = -1
    best_ind = None
    best_sched = None

    for gen in range(generations):
        fits = []
        scheds = []
        for indiv in pop:
            fit, sched = decode_permutation(indiv)
            fits.append(fit)
            scheds.append(sched)
            if fit > best_fit:
                best_fit = fit
                best_ind = indiv[:]
                best_sched = sched
        # reporting
        if (gen % 10 == 0) or gen == generations-1:
            print(f"Gen {gen:3d} best_completed so far = {best_fit}")
        # next generation
        new_pop = []
        # elitism: keep best individual
        elite_idx = max(range(len(pop)), key=lambda i: fits[i])
        new_pop.append(pop[elite_idx][:])
        while len(new_pop) < pop_size:
            p1 = tournament_select(pop, fits)
            p2 = tournament_select(pop, fits)
            if random.random() < 0.85:
                c1, c2 = pmx_crossover(p1, p2)
            else:
                c1, c2 = p1[:], p2[:]
            c1 = swap_mutation(c1, rate=0.18)
            c2 = swap_mutation(c2, rate=0.18)
            new_pop.append(c1)
            if len(new_pop) < pop_size:
                new_pop.append(c2)
        pop = new_pop

    return best_fit, best_ind, best_sched

# ---------------------- RUN DEMO ----------------------
if __name__ == "__main__":
    random.seed(1234)
    start_t = time.time()
    best_fit, best_ind, best_sched = run_ga(pop_size=500, generations=60)
    elapsed = time.time() - start_t
    print(f"\nGA finished in {elapsed:.1f}s. Best completed tires = {best_fit} / {NUM_OVENS}")
    print("Best permutation (first 32 genes):")
    print(best_ind)

    # hourly summary (in minutes) or per 600-second buckets
    completed_ends = [info['oven_end'] for info in best_sched.values() if info.get('assigned')]
    completed_ends = [t for t in completed_ends if t <= HORIZON]
    completed_ends.sort()
    # group into 24 hourly buckets (by seconds)
    hourly = [0]*24
    for t in completed_ends:
        h = min(23, t // 3600)
        hourly[h] += 1
    print("\nHourly completed (by hour):")
    for hr in range(24):
        print(f"Hour {hr:02d}:00 - {hourly[hr]} tires")

    # show sample of scheduled jobs
    sample = sorted([(jid, info['oven_start'], info['oven_end']) for jid,info in best_sched.items() if info.get('assigned')], key=lambda x: x[1])[:100]
    print("\nSample schedule (oven_id, oven_start, oven_end) - showing first 32:")
    for row in sample:
        print(row)


Initial population size: 30
Gen   0 best_completed so far = 32
Gen  10 best_completed so far = 32
Gen  20 best_completed so far = 32
Gen  30 best_completed so far = 32
Gen  40 best_completed so far = 32
Gen  50 best_completed so far = 32
Gen  59 best_completed so far = 32

GA finished in 29.3s. Best completed tires = 32 / 32
Best permutation (first 32 genes):
[1, 17, 2, 18, 3, 19, 4, 20, 5, 21, 6, 22, 7, 23, 8, 24, 9, 25, 10, 26, 11, 27, 12, 28, 13, 29, 14, 30, 15, 31, 16, 32]

Hourly completed (by hour):
Hour 00:00 - 19 tires
Hour 01:00 - 13 tires
Hour 02:00 - 0 tires
Hour 03:00 - 0 tires
Hour 04:00 - 0 tires
Hour 05:00 - 0 tires
Hour 06:00 - 0 tires
Hour 07:00 - 0 tires
Hour 08:00 - 0 tires
Hour 09:00 - 0 tires
Hour 10:00 - 0 tires
Hour 11:00 - 0 tires
Hour 12:00 - 0 tires
Hour 13:00 - 0 tires
Hour 14:00 - 0 tires
Hour 15:00 - 0 tires
Hour 16:00 - 0 tires
Hour 17:00 - 0 tires
Hour 18:00 - 0 tires
Hour 19:00 - 0 tires
Hour 20:00 - 0 tires
Hour 21:00 - 0 tires
Hour 22:00 - 0 tires
Hour

In [20]:
# tire_mip.py
from pulp import LpProblem, LpVariable, LpMaximize, lpSum, LpStatus, PULP_CBC_CMD, LpBinary, LpContinuous, value
import math
from itertools import combinations
from typing import Dict, Tuple

# ---------- copy your data from tire_ga.py ----------
HORIZON = 24 * 3600
CURING_TIME = 3000
NUM_OVENS = 32
OVENS_PER_ROW = 16
NUM_HOISTS = 2
NUM_MILLS = 3
NUM_DEVICES = 5
HOIST_OP_DURATION = 6

OVEN_INFO = {
    1:  ('A', 10),  2: ('C', 12),  3: ('G', 14),  4: ('E', 16),
    5:  ('B', 18),  6: ('D', 20),  7: ('F', 22),  8: ('H', 24),
    9:  ('A', 26), 10: ('D', 28), 11: ('G', 30), 12: ('F', 32),
    13: ('H', 34), 14: ('G', 36), 15: ('F', 38), 16: ('A', 40),
    17: ('E', 10), 18: ('D', 12), 19: ('E', 14), 20: ('A', 16),
    21: ('G', 18), 22: ('G', 20), 23: ('D', 22), 24: ('B', 24),
    25: ('F', 26), 26: ('H', 28), 27: ('A', 30), 28: ('E', 32),
    29: ('B', 34), 30: ('H', 36), 31: ('D', 38), 32: ('G', 40),
}

MILL_TIMES = {
    'A': (21, 19, 28),
    'B': (18, 20, 22),
    'C': (26, 24, 31),
    'D': (20, 23, 26),
    'E': (24, 23, 30),
    'F': (15, 13, 17),
    'G': (17, 18, 22),
    'H': (31, 26, 38)
}

SEG_TIME = {
    'M1_M2': 5,
    'M2_M3': 4,
    'M3_CART': 8,
    'CART_M1': 3,
}

MOVE_INTERNAL = {'after_m1':0, 'after_m2':0, 'after_m3':0}

JOBS = list(range(1, NUM_OVENS+1))
T = HORIZON
# Big-M (conservative)
M = T + 100000

# Prepare job parameters
p = {}  # p[j][m] processing times
trav = {}
for j in JOBS:
    pid, travel = OVEN_INFO[j]
    r1,r2,r3 = MILL_TIMES[pid]
    p[j] = {1: r1, 2: r2, 3: r3}
    trav[j] = travel

# ---------- BUILD MODEL ----------
prob = LpProblem("Tire_Scheduling_MaxCompleted", LpMaximize)

# Start times
S1 = {j: LpVariable(f"S1_{j}", lowBound=0, upBound=T, cat=LpContinuous) for j in JOBS}
S2 = {j: LpVariable(f"S2_{j}", lowBound=0, upBound=T, cat=LpContinuous) for j in JOBS}
S3 = {j: LpVariable(f"S3_{j}", lowBound=0, upBound=T, cat=LpContinuous) for j in JOBS}
S_m1m2 = {j: LpVariable(f"S_m1m2_{j}", lowBound=0, upBound=T, cat=LpContinuous) for j in JOBS}
S_m2m3 = {j: LpVariable(f"S_m2m3_{j}", lowBound=0, upBound=T, cat=LpContinuous) for j in JOBS}
S_cart = {j: LpVariable(f"S_cart_{j}", lowBound=0, upBound=T, cat=LpContinuous) for j in JOBS}
S_hoist = {j: LpVariable(f"S_hoist_{j}", lowBound=0, upBound=T, cat=LpContinuous) for j in JOBS}
S_oven = {j: LpVariable(f"S_oven_{j}", lowBound=0, upBound=T+M, cat=LpContinuous) for j in JOBS}
E_oven = {j: LpVariable(f"E_oven_{j}", lowBound=0, upBound=T+M, cat=LpContinuous) for j in JOBS}

# Completed binary
z = {j: LpVariable(f"z_{j}", cat=LpBinary) for j in JOBS}

# Ordering binaries for resources (mills and segments and hoists)
# For each unordered pair i<j we create x_ij^r and x_ji^r by convention we only store x_ij and use x_ji = 1-x_ij
# But for clarity we use both x_ij and x_ji explicitly here.

resources = []
# define resource name -> (start_var_dict, duration_func(job))
resources.append(("mill1", S1, lambda j: p[j][1]))
resources.append(("mill2", S2, lambda j: p[j][2]))
resources.append(("mill3", S3, lambda j: p[j][3]))
resources.append(("seg_m1m2", S_m1m2, lambda j: SEG_TIME['M1_M2']))
resources.append(("seg_m2m3", S_m2m3, lambda j: SEG_TIME['M2_M3']))
resources.append(("seg_m3cart", S_cart, lambda j: SEG_TIME['M3_CART']))
# CART_M1 is used for device return; we create a variable but it's used only for device return sequencing
resources.append(("seg_cartm1", None, lambda j: SEG_TIME['CART_M1']))  # start var to be created per-job if needed

# ordering binaries: dict of dict keyed by resource -> (i,j)
order = {}
for rname, startvar, durfunc in resources:
    order[rname] = {}
    for (i,j) in combinations(JOBS, 2):
        order[rname][(i,j)] = LpVariable(f"ord_{rname}_{i}_{j}", cat=LpBinary)
        # ord_{i,j} = 1 means i before j on that resource

# Hoist ordering: only for jobs whose hoist_idx same
def hoist_idx_of_job(j):
    return 0 if j <= OVENS_PER_ROW else 1

hoist_order = {}
for h in range(NUM_HOISTS):
    hoist_order[h] = {}
    # pairs where both jobs use hoist h
    jobs_h = [j for j in JOBS if hoist_idx_of_job(j)==h]
    for (i,j) in combinations(jobs_h, 2):
        hoist_order[h][(i,j)] = LpVariable(f"ord_hoist{h}_{i}_{j}", cat=LpBinary)

# ---------- Precedence constraints for each job ----------
for j in JOBS:
    # Mill1 start >= 0 implicitly by variable lower bound
    prob += S_m1m2[j] >= S1[j] + p[j][1] + MOVE_INTERNAL['after_m1']
    prob += S2[j] >= S_m1m2[j] + SEG_TIME['M1_M2']
    prob += S_m2m3[j] >= S2[j] + p[j][2] + MOVE_INTERNAL['after_m2']
    prob += S3[j] >= S_m2m3[j] + SEG_TIME['M2_M3']
    prob += S_cart[j] >= S3[j] + p[j][3] + MOVE_INTERNAL['after_m3']
    # hoist may start at or after device arrival
    prob += S_hoist[j] >= S_cart[j]
    # oven insert after hoist op and oven travel
    prob += S_oven[j] >= S_hoist[j] + HOIST_OP_DURATION + trav[j]
    # oven end:
    prob += E_oven[j] == S_oven[j] + CURING_TIME
    # link z_j: if z_j==1 then E_oven[j] <= T, else can be >T
    prob += E_oven[j] <= T + M*(1 - z[j])
    # ensure S_oven reasonable if z=1:
    prob += S_oven[j] <= T + M*(1 - z[j])

# ---------- Non-overlap (disjunctive) constraints ----------
# For each resource r (where startvar exists), for each pair i<j:
for rname, startvar_dict, durfunc in resources:
    if startvar_dict is None:
        # seg_cartm1 not used as primary start var for jobs in current simplified form
        continue
    dur_i = lambda j: durfunc(j)
    dur_j = lambda j: durfunc(j)
    for (i,j) in combinations(JOBS, 2):
        x_ij = order[rname][(i,j)]
        # i before j
        prob += startvar_dict[i] + durfunc(i) <= startvar_dict[j] + M * (1 - x_ij)
        # j before i
        prob += startvar_dict[j] + durfunc(j) <= startvar_dict[i] + M * x_ij
        # symmetry broken implicitly: x_ij binary handles both orders

# ---------- Hoist non-overlap (only jobs mapping to same hoist) ----------
for h in range(NUM_HOISTS):
    for (i,j), var in hoist_order[h].items():
        # hoist operation occupies [S_hoist, S_hoist + HOIST_OP_DURATION)
        prob += S_hoist[i] + HOIST_OP_DURATION <= S_hoist[j] + M * (1 - var)
        prob += S_hoist[j] + HOIST_OP_DURATION <= S_hoist[i] + M * var

# ---------- Optional: Oven resource exclusivity (not needed if each job is tied to unique oven) ----------
# If you plan to allow multiple loads into same oven id, add ovens occupancy constraints.
# Here each job j corresponds to a specific oven id so no cross-job oven conflict (skip).

# ---------- Objective: maximize completed jobs ----------
prob += lpSum([z[j] for j in JOBS]), "MaxCompletedTires"

# ---------- Solve ----------
# If you have gurobi, prefer to call gurobi from PuLP or write direct gurobipy model.
solver = PULP_CBC_CMD(msg=True, timeLimit=600)  # timeLimit sec (optional)
prob.solve(solver)

print("Status:", LpStatus[prob.status])
print("Objective (completed):", value(prob.objective))
# collect schedule
schedule = {}
for j in JOBS:
    schedule[j] = {
        'S1': value(S1[j]),
        'S2': value(S2[j]),
        'S3': value(S3[j]),
        'S_oven': value(S_oven[j]),
        'E_oven': value(E_oven[j]),
        'z': int(value(z[j])),
        'hoist_idx': 0 if j <= OVENS_PER_ROW else 1
    }

# print sample completed
completed = [j for j in JOBS if schedule[j]['z']==1]
print("Completed jobs:", len(completed), completed)


Status: Optimal
Objective (completed): 32.0
Completed jobs: 32 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
